# writers

The Midgard **writers** module can be used for writing files. For example following file formats are available via Midgard:

- Bernese station information files (*.ABB, *.CLU, *.CRD, *.STA, *.VEL)
- GipsyX site information file
- GAMIT station information file
- ...

All available writers can be shown with `names` method:

In [1]:
# Import writers package
from midgard import writers

# List all available writers
writers.names()

['bernese_abb',
 'bernese_clu',
 'bernese_crd',
 'bernese_sta',
 'bernese_vel',
 'gamit_apr_eq',
 'gamit_station_info',
 'gipsyx_site_info',
 'sinex_tms']

## Use of station information file writers 

Following station information file writers exists:

| Writer name        | Description |
|:------------------ | :-----------|
| bernese_abb        | Bernese station abbreviation table in *.ABB format |
| bernese_clu        | Bernese station list file in *.CLU format |
| bernese_crd        | Bernese station coordinate file in *.CRD format | 
| bernese_sta        | Bernese station information file in *.STA format |
| bernese_vel        | Bernese station velocity file in *.VEL format |
| gamit_apr_eq       | GAMIT station coordinates files in *.apr and *.eq format |
| gamit_station_info | GAMIT station information file |
| gipsyx_site_info   | GipsyX station information file |

As a first step the input data has to be generated for the writers. That means station information has to be collected from one or more of the following sources:

| Source             | Parser name | Description |
|:------------------ | :-----------| :-----------|
| SINEX              | sinex_site  | Station information given in SINEX format files, which can inlude information about antenna, receiver, eccentricity and station coordinates (like https://files.igs.org/pub/station/general/igs.snx) | 
| SSC                | ssc_site    |Station coordinate and velocity is given in SSC format files (like https://itrf.ign.fr/docs/solutions/itrf2014/ITRF2014_GNSS.SSC.txt) |
| M3G                | m3g         | M3G application programming interface (API) from https://gnss-metadata.eu/site/api-docs. The M3G is a metadata management and distribution system for multiple GNSS networks (https://gnss-metadata.eu), which is accessable via M3G application programming interface (API).  |

Midgard **parsers** can be used to read SSC/SINEX files and the **site_info** module to access the M3G API. In the following a SSC and a SINEX file are read as **source_data**. These **source_data** are applied to initialize a Midgard **site_info** object. Midgard **site_info** class provides a data structure to save relevant station information. 

In [10]:
# Standard library import
from datetime import datetime
from pprint import pprint # Used to print variables more pretty

# Import Midgard parsers
from midgard import parsers

# Import SiteCoord class
from midgard.site_info.site_info import SiteInfo

# Read station information (antenna, receiver, eccentricity) from SINEX file
p_snx = parsers.parse_file(
            parser_name="sinex_site", 
            file_path="../tests/parsers/example_files/sinex_site",
)

# Read station coordinates/velocities from SSC file
p_ssc = parsers.parse_file(
            parser_name="ssc_site", 
            file_path="../tests/parsers/example_files/ssc_site",
)

# Get station dictionary of SiteInfo instance for SINEX source
site_info = SiteInfo.get(
            source="snx", # File type specifier: snx = SINEX; ssc = SSC
            source_data=p_snx.as_dict(),
            stations=p_snx.as_dict().keys(), # List with station identifiers
            source_path=p_snx.file_path,
)

# Get station dictionary of SiteInfo instance for SSC source
site_info_ssc = SiteInfo.get(
            source="ssc", # File type specifier: snx = SINEX; ssc = SSC
            source_data=p_ssc.as_dict(),
            stations=p_ssc.as_dict().keys(), # List with station identifiers
            source_path=p_ssc.file_path,
)

# Show site_info dictionary
pprint(site_info)

{'brux': {'antenna': AntennaHistorySinex(station='brux'),
          'eccentricity': EccentricityHistorySinex(station='brux'),
          'identifier': IdentifierSinex(station='brux', ),
          'receiver': ReceiverHistorySinex(station='brux'),
          'site_coord': SiteCoordHistorySinex(station='brux')},
 'pots': {'antenna': AntennaHistorySinex(station='pots'),
          'eccentricity': EccentricityHistorySinex(station='pots'),
          'identifier': IdentifierSinex(station='pots', ),
          'receiver': ReceiverHistorySinex(station='pots'),
          'site_coord': SiteCoordHistorySinex(station='pots')},
 'zimm': {'antenna': AntennaHistorySinex(station='zimm'),
          'eccentricity': EccentricityHistorySinex(station='zimm'),
          'identifier': IdentifierSinex(station='zimm', ),
          'receiver': ReceiverHistorySinex(station='zimm'),
          'site_coord': SiteCoordHistorySinex(station='zimm')}}


The SINEX files like **igs.snx** or **euref.snx** include only approximated station coordinates. Therefore we read in addition the SSC file with precise coordinates and velocities. The SINEX **site_info** dictionary will now be updated with the **site_coord** objects of the SSC **site_info_ssc** dictionary.

In [11]:
# Update SINEX source with station coordinates/velocities from SSC source
for sta in site_info.keys():
    site_info[sta]["site_coord"] = site_info_ssc[sta]["site_coord"]
    
# Show site_info dictionary
pprint(site_info)

{'brux': {'antenna': AntennaHistorySinex(station='brux'),
          'eccentricity': EccentricityHistorySinex(station='brux'),
          'identifier': IdentifierSinex(station='brux', ),
          'receiver': ReceiverHistorySinex(station='brux'),
          'site_coord': SiteCoordHistorySsc(station='brux')},
 'pots': {'antenna': AntennaHistorySinex(station='pots'),
          'eccentricity': EccentricityHistorySinex(station='pots'),
          'identifier': IdentifierSinex(station='pots', ),
          'receiver': ReceiverHistorySinex(station='pots'),
          'site_coord': SiteCoordHistorySsc(station='pots')},
 'zimm': {'antenna': AntennaHistorySinex(station='zimm'),
          'eccentricity': EccentricityHistorySinex(station='zimm'),
          'identifier': IdentifierSinex(station='zimm', ),
          'receiver': ReceiverHistorySinex(station='zimm'),
          'site_coord': SiteCoordHistorySsc(station='zimm')}}


The last step is to use the **site_info** dictionary for writing the station information in different formats with the writers like **bernese_crd**, **bernese_sta**, **gamit_station_info** or **gipsyx_site_info**.

In [12]:
# Import writer
from midgard.writers.bernese_crd import bernese_crd
from midgard.writers.bernese_sta import bernese_sta
from midgard.writers.bernese_vel import bernese_vel
from midgard.writers.gipsyx_site_info import gipsyx_site_info

# Use bernese_crd writer
bernese_crd(
    file_path="./examples/writers/BERNESE.CRD",
    site_info=site_info,
    datum="IGb14", # Reference system of station coordinates
    epoch=datetime(2010,1,1), # Reference epoch of station coordinates
    agency="NMA",
)

# Display output file
from IPython.display import Code
Code(filename='./examples/writers/BERNESE.CRD', language='python')

NMA solution 20240307                                            07-MAR-24 11:58
--------------------------------------------------------------------------------
LOCAL GEODETIC DATUM: IGb14             EPOCH: 2010-01-01 00:00:00

NUM  STATION NAME           X (M)          Y (M)          Z (M)     FLAG

  1  BRUX 13101M010    4027881.51500   306998.57700  4919498.91700    A
  2  POTS 14106M003    3800689.55300   882077.46500  5028791.37000    A
  3  ZIMM 14001M004    4331296.99600   567555.96700  4633133.99300    A

In [5]:
# Use bernese_vel writer
bernese_vel(
    file_path="./examples/writers/BERNESE.VEL",
    site_info=site_info,
    datum="IGb14", # Reference system of station velocities
    agency="NMA",
)

# Display output file
from IPython.display import Code
Code(filename='./examples/writers/BERNESE.VEL', language='python')

NMA solution 20240307                                            07-MAR-24 11:52
--------------------------------------------------------------------------------
LOCAL GEODETIC DATUM: IGb14             

NUM  STATION NAME           VX (M/Y)       VY (M/Y)       VZ (M/Y)  FLAG   PLATE

  1  BRUX 13101M010         -0.01370        0.01690        0.01070    A        
  2  POTS 14106M003         -0.01620        0.01600        0.00930    A        
  3  ZIMM 14001M004         -0.01390        0.01800        0.01180    A

In [6]:
# Use bernese_sta writer
bernese_sta(
    file_path="./examples/writers/BERNESE.STA",
    site_info=site_info,
    agency="NMA", # Agency which uses this file for processing
)

# Display output file
from IPython.display import Code
Code(filename='./examples/writers/BERNESE.STA', language='python')

BERNESE V.5.2 STA FILE FOR NMA PROCESSING                        07-Mar-24 11:52
--------------------------------------------------------------------------------

FORMAT VERSION: 1.01
TECHNIQUE:      GNSS

TYPE 001: RENAMING OF STATIONS
------------------------------

STATION NAME          FLG          FROM                   TO         OLD STATION NAME      REMARK
****************      ***  YYYY MM DD HH MM SS  YYYY MM DD HH MM SS  ********************  ************************
BRUX 13101M010        001  2006 07 07 00 00 00  9999 12 31 23 59 59  BRUX                  From sinex_site file
POTS 14106M003        001  1994 10 28 00 00 00  9999 12 31 23 59 59  POTS                  From sinex_site file
ZIMM 14001M004        001  1993 05 01 00 00 00  9999 12 31 23 59 59  ZIMM                  From sinex_site file


TYPE 002: STATION INFORMATION
-----------------------------

STATION NAME          FLG          FROM                   TO         RECEIVER TYPE         RECEIVER SERIAL NBR   REC #   ANTENNA TYPE          ANTENNA SERIAL NBR    ANT #    NORTH      EAST      UP      DESCRIPTION             REMARK
****************      ***  YYYY MM DD HH MM SS  YYYY MM DD HH MM SS  ********************  ********************  ******  ********************  ********************  ******  ***.****  ***.****  ***.****  **********************  ************************
BRUX 13101M010        001  2006 07 07 00 00 00  2008 02 15 08 00 00  SEPT POLARX2                          1436    1436  ASH701945E_M    NONE                999999  999999    0.0010    0.0000    0.1266  Brussels                2.6.2
BRUX 13101M010        001  2008 02 15 08 00 00  2008 03 19 09 00 00  ASHTECH Z-XII3T                      RT820     820  ASH701945E_M    NONE                999999  999999    0.0010    0.0000    0.1266  Brussels                1L01-1D04
BRUX 13101M010        001  2008 03 19 09 00 00  2008 06 17 09 00 00  ASHTECH Z-XII3T                      RT820     820  ASH701945C_M    NONE                999999  999999    0.0010    0.0000    0.1266  Brussels                1L01-1D04
BRUX 13101M010        001  2008 06 17 09 00 00  2008 09 02 13 00 00  ASHTECH Z-XII3T                      RT820     820  ASH701945C_M    NONE                999999  999999    0.0010    0.0000    0.1286  Brussels                1L01-1D04
BRUX 13101M010        001  2008 09 02 13 00 00  2008 09 22 09 00 00  ASHTECH Z-XII3T                      RT820     820  ASH701945C_M    NONE                999999  999999    0.0010    0.0000    0.1266  Brussels                1L01-1D04
BRUX 13101M010        001  2008 09 22 09 00 00  2008 10 02 09 30 00  ASHTECH Z-XII3T                      RT820     820  TRM55971.00     NONE                999999  999999    0.0010    0.0000    0.1266  Brussels                1L01-1D04
BRUX 13101M010        001  2008 10 02 09 30 00  2008 10 20 06 30 00  ASHTECH Z-XII3T                      RT820     820  TRM55971.00     TZGD                999999  999999    0.0010    0.0000    0.1266  Brussels                1L01-1D04
BRUX 13101M010        001  2008 10 20 06 30 00  2010 06 28 13 55 00  ASHTECH Z-XII3T                      RT820     820  ASH701945C_M    NONE                999999  999999    0.0010    0.0000    0.1266  Brussels                1L01-1D04
BRUX 13101M010        001  2010 06 28 13 55 00  2011 03 07 12 00 00  SEPT POLARX3ETR                      20010   20010  ASH701945C_M    NONE                999999  999999    0.0010    0.0000    0.1266  Brussels                1.4.0
BRUX 13101M010        001  2011 03 07 12 00 00  2012 01 31 14 00 00  SEPT POLARX3ETR                      20010   20010  JAVRINGANT_DM   NONE                999999  999999    0.0010    0.0000    0.4689  Brussels                1.4.0
BRUX 13101M010        001  2012 01 31 14 00 00  2012 03 05 16 00 00  SEPT POLARX4TR                       30013   30013  JAVRINGANT_DM   NONE                999999  999999    0.0010    0.0000    0.4689  Brussels                2.3.3
BRUX 13101M010        001  2012 03 05 16 

In [7]:
# Use gipsyx_site_info writer
gipsyx_site_info(
    file_path="./examples/writers/gipsyx_site_info",
    site_info=site_info,
)

# Display output file
from IPython.display import Code
Code(filename='./examples/writers/gipsyx_site_info', language='python')

KEYWORDS: ANT END ID POSTSEISMIC RX STATE
BRUX  ID  13101M010  Brussels, Belgium
BRUX  ANT    2006-07-07 00:00:00  ASH701945E_M NONE   0.000000e+00   1.000000e-03   1.266000e-01  # CR620
BRUX  ANT    2008-03-19 09:00:00  ASH701945C_M NONE   0.000000e+00   1.000000e-03   1.266000e-01  # CR620
BRUX  ANT    2008-06-17 09:00:00  ASH701945C_M NONE   0.000000e+00   1.000000e-03   1.286000e-01  # CR620
BRUX  ANT    2008-09-02 13:00:00  ASH701945C_M NONE   0.000000e+00   1.000000e-03   1.266000e-01  # CR620
BRUX  ANT    2008-09-22 09:00:00  TRM55971.00 NONE   0.000000e+00   1.000000e-03   1.266000e-01  # 30765
BRUX  ANT    2008-10-02 09:30:00  TRM55971.00 TZGD   0.000000e+00   1.000000e-03   1.266000e-01  # 30765
BRUX  ANT    2008-10-20 06:30:00  ASH701945C_M NONE   0.000000e+00   1.000000e-03   1.266000e-01  # CR620
BRUX  ANT    2011-03-07 12:00:00  JAVRINGANT_DM NONE   0.000000e+00   1.000000e-03   4.689000e-01  # 00464
BRUX  RX     2006-07-07 00:00:00  SEPT POLARX2 # 1436 2.6.2
BRUX  RX     2008-02-15 08:00:00  ASHTECH Z-XII3T # RT820 1L01-1D04
BRUX  RX     2010-06-28 13:55:00  SEPT POLARX3ETR # 20010 1.4.0
BRUX  RX     2012-01-31 14:00:00  SEPT POLARX4TR # 30013 2.3.3
BRUX  RX     2012-03-05 16:00:00  SEPT POLARX4TR # 30013 2.3-tst1202
BRUX  RX     2012-09-24 11:45:00  SEPT POLARX4TR # 30013 2.3.4
BRUX  RX     2013-02-04 13:55:00  SEPT POLARX4TR # 30013 2.3.4
BRUX  RX     2014-03-17 10:40:00  SEPT POLARX4TR # 30013 2.5.2
BRUX  RX     2015-09-07 14:50:00  SEPT POLARX4TR # 30013 2.9.0
BRUX  RX     2016-10-24 09:00:00  SEPT POLARX4TR # 30013 2.9.5
BRUX  RX     2017-01-03 12:15:00  SEPT POLARX4TR # 30013 2.9.6
BRUX  RX     2017-03-20 00:00:00  SEPT POLARX4TR # 30013 2.9.6
BRUX  RX     2018-11-12 08:31:00  SEPT POLARX4TR # 30013 2.9.6-extre
BRUX  RX     2020-02-25 13:30:00  SEPT POLARX5TR # 30576 5.3.2
BRUX  STATE  2012-02-10 00:00:00    4.027881514e+06        3.069985780e+05        4.919498918e+06       -1.370000000e-02        1.690000000e-02        1.070000000e-02
BRUX  STATE  2012-03-28 00:00:00    4.027881515e+06        3.069985770e+05        4.919498917e+06       -1.370000000e-02        1.690000000e-02        1.070000000e-02
POTS  ID  14106M003  Potsdam, Germany
POTS  ANT    1994-10-28 00:00:00  AOAD/M_T NONE   0.000000e+00   0.000000e+00   4.600000e-02  # 261
POTS  ANT    1995-10-03 08:00:00  AOAD/M_T NONE   0.000000e+00   0.000000e+00   4.600000e-02  # 235
POTS  ANT    2009-04-15 13:15:00  AOAD/M_T NONE   0.000000e+00   0.000000e+00   4.600000e-02  # 354-U
POTS  ANT    2011-02-15 17:00:00  JAV_RINGANT_G3T NONE   0.000000e+00   0.000000e+00   1.206000e-01  # 316
POTS  ANT    2018-07-17 10:00:00  JAVRINGANT_G5T NONE   0.000000e+00   0.000000e+00   1.206000e-01  # 6077C
POTS  RX     1994-10-01 00:00:00  ROGUE SNR-8000 # 289 2.8.32.1X
POTS  RX     1996-01-16 13:48:00  ROGUE SNR-8000 # 279 3.2
POTS  RX     1996-05-30 08:01:00  ROGUE SNR-8000 # 289 3.2
POTS  RX     1999-08-20 00:00:00  AOA SNR-8000 ACT # 281 3.2.32.9
POTS  RX     2000-08-20 00:00:00  AOA SNR-8000 ACT # 281-U 3.3.32.3
POTS  RX     2009-03-30 00:00:00  SEPT POLARX2 # 1358 2.5.0
POTS  RX     2011-02-15 17:00:00  JAVAD TRE_G3TH DELTA # 205 3.1.7
POTS  RX     2011-11-03 14:30:00  JAVAD TRE_G3TH DELTA # 205 3.2.7
POTS  RX     2011-12-20 10:38:00  JAVAD TRE_G3TH DELTA # 205 3.3.5
POTS  RX     2012-06-12 09:00:00  JAVAD TRE_G3TH DELTA # 205 3.4.4b0
POTS  RX     2013-01-09 10:13:00  JAVAD TRE_G3TH DELTA # 205 3.4.7
POTS  RX     2015-09-15 13:58:00  JAVAD TRE_G3TH DELTA # 205 3.5.10
POTS  RX     2016-10-25 14:06:00  JAVAD TRE_G3TH DELTA # 205 3.6.7
POTS  RX     2018-07-09 14:48:00  JAVAD TRE_G3TH DELTA # 205 3.7.3P1
POTS  RX     2018-07-17 10:00:00  JAVAD TRE_3 # 41717 3.7.3P1
POTS  RX     2018-09-06 10:00:00  JAVAD TRE_3 # 41717 3.7.4b0-193
POTS  RX     2018-12-11 08:36:00  JAVAD TRE_3 # 41717 3.7.5
POTS  RX     2019-01-31 09:40:00  JAVAD TRE_3 # 41717 3.7.5p1
POTS  RX     2019-04-09 12:22:00  JAVAD TRE_3 # 41717 3.7.6
POTS  RX     2019-12-20 11:19:00  JAVAD TRE_3 # 41717 3.7.

## Use of SINEX timeseries writer sinex_tms

An example is shown, how to use the SINEX timeseries writer **sinex_tms** module. As a first step the input data has to be generated. In this example several daily station coordinate solutions in SINEX format are read to generate the needed Midgard dataset as input for the **sinex_tms** writer.

In [1]:
# Standard library import
from datetime import datetime, timedelta

# Import Midgard parsers
import midgard
from midgard.data.dataset import Dataset
from midgard import parsers

# Initialize Midgard dataset
dset = Dataset()

# Define time period
date_from = datetime(2023, 6, 9)
date_to = datetime(2023, 6, 11)

# Loop over time period
date = date_from
while date <= date_to:
    
    # Read SINEX file with station coordinate solutions
    yyddd = date.strftime("%y%j")
    p = parsers.parse_file(
                parser_name="sinex_site", 
                file_path=f"./examples/parsers/F1_{yyddd}0.SNX",
    )
    
    if date_from == date:
        # Initialize first dataset entry
        dset = p.as_dataset()
    else:
        # Add following daily SINEX file solution 
        dset.extend(p.as_dataset())
    
    # Increase date counter
    date += timedelta(days=1)

The next step is to use the generated dataset **dset** for writing SINEX timeseries files for each station.

In [3]:
# Import writer
from midgard.writers.sinex_tms import sinex_tms

# Loop over stations
for sta in dset.unique("station"):

    # Use sinex_tms writer
    sinex_tms(
        dset=dset,
        station=sta,
        file_path=f"./examples/writers/{sta}.tms",
        contact="dummy@kartverket.no",
        data_agency="NMA",
        file_agency="NMA",
        input_="Daily GNSS Solutions processed with Bernese 5.4",
        organization="Norwegian Mapping Authority",
        software=f"Midgard {midgard.__version__}",
        version="001",
    )

# Display output file
from IPython.display import Code
Code(filename='./examples/writers/brux.tms', language='python')

%=TMS 1.0 NMA 2024:068:58110 NMA 2023:160:43185 2023:162:43185 P BRUX
+FILE/REFERENCE
*INFO_TYPE________: INFO___________________________________________________________________________________
 DESCRIPTION        Norwegian Mapping Authority
 OUTPUT             GNSS station coordinate timeseries solution                 
 CONTACT            dummy@kartverket.no                                         
 SOFTWARE           Midgard 1.3.0                                               
 INPUT              Daily GNSS Solutions processed with Bernese 5.4             
 VERSION NUMBER     001                                                         
-FILE/REFERENCE
+TIMESERIES/COLUMNS
*__COL __NAME______________ __UNIT______________ __DESCRIPTION__________________________________________________
     1 YYYY-MM-DD                                Date in format year, month and day (e.g. 2023-06-01)
     2 YEAR                 y                    Date as decimal year (2023.4137)
     3 X                    m                    X-coordinate of geocentric site coordinates
     4 Y                    m                    Y-coordinate of geocentric site coordinates
     5 Z                    m                    Z-coordinate of geocentric site coordinates
     6 SIG_X                m                    Standard deviation of geocentric X-coordinate
     7 SIG_Y                m                    Standard deviation of geocentric Y-coordinate
     8 SIG_Z                m                    Standard deviation of geocentric Z-coordinate
-TIMESERIES/COLUMNS
+TIMESERIES/DATA
* _YYYY-MM-DD _YEAR______ _X___________ _Y___________ _Z___________ _SIG_X___ _SIG_Y___ _SIG_Z___
 2023-06-09    2023.43699  4027881.3340   306998.8067  4919499.0515    0.0007    0.0003    0.0008
 2023-06-10    2023.43973  4027881.3328   306998.8066  4919499.0520    0.0007    0.0003    0.0009
 2023-06-11    2023.44247  4027881.3304   306998.8074  4919499.0505    0.0008    0.0003    0.0010
-TIMESERIES/DATA